In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv


--2025-09-30 20:09:46--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv.2’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.008s  

2025-09-30 20:09:46 (10.2 MB/s) - ‘course_lead_scoring.csv.2’ saved [80876/80876]



In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [3]:
df = pd.read_csv("course_lead_scoring.csv")
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [4]:
df.isna().sum()


,0
lead_source,128
industry,134
number_of_courses_viewed,0
annual_income,181
employment_status,100
location,63
interaction_count,0
lead_score,0
converted,0


In [5]:
numeric_columns = df.select_dtypes(include=['int64', 'float64'])
categorical_columns = df.select_dtypes(include=['object'])

In [6]:
df[numeric_columns.columns] = df[numeric_columns.columns].fillna(0.0)
df[categorical_columns.columns] = df[categorical_columns.columns].fillna('NA')

In [7]:
df

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NA,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NA,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1
...,...,...,...,...,...,...,...,...,...
1457,referral,manufacturing,1,0.0,self_employed,north_america,4,0.53,1
1458,referral,technology,3,65259.0,student,europe,2,0.24,1
1459,paid_ads,technology,1,45688.0,student,north_america,3,0.02,1
1460,referral,NA,5,71016.0,self_employed,north_america,0,0.25,1


In [8]:
df['industry'].value_counts().sort_values(ascending = False)

,count
industry,
retail,203
finance,200
other,198
healthcare,187
education,187
technology,179
manufacturing,174
NA,134


In [9]:
numeric_columns.corr()

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
number_of_courses_viewed,1.000000,0.031551,-0.023565,-0.004879,0.435914
annual_income,0.031551,1.000000,0.048618,0.005334,0.078256
interaction_count,-0.023565,0.048618,1.000000,0.009888,0.374573
lead_score,-0.004879,0.005334,0.009888,1.000000,0.193673
converted,0.435914,0.078256,0.374573,0.193673,1.000000


In [10]:
y = df['converted']
del df['converted']

In [11]:
X_train,X_test, y_train,y_test = train_test_split(df,y,test_size=0.2,random_state=42)

In [12]:
X_train,X_val, y_train,y_val = train_test_split(X_train,y_train,test_size=0.25,random_state=42)

In [13]:
len(X_train), len(X_val), len(X_test)

(876, 293, 293)

In [32]:
from sklearn.metrics import mutual_info_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import RidgeClassifier

In [15]:
for col in X_train.columns:
  result = round(mutual_info_score(y_train, X_train[col]),2)
  print(col, result)

lead_source 0.04
industry 0.01
number_of_courses_viewed 0.11
annual_income 0.58
employment_status 0.01
location 0.0
interaction_count 0.08
lead_score 0.08


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/cluster/_supervised.py:59: UserWarning: Clustering metrics expects discrete values but received binary values for label, and continuous values for target
  warnings.warn(msg, UserWarning)


In [16]:
dv = DictVectorizer(sparse=False)

train_dict = X_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = X_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

In [17]:

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)


In [18]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [19]:
y_pred = model.predict(X_val)
orig_accuracy = (y_val == y_pred).mean()
print(round(orig_accuracy,2))

0.7


In [22]:
for col in df.columns:
  X_train,X_test, y_train,y_test = train_test_split(df,y,test_size=0.2,random_state=42)
  X_train,X_val, y_train,y_val = train_test_split(X_train,y_train,test_size=0.25,random_state=42)
  dv = DictVectorizer(sparse=False)
  train_dict = X_train.loc[:, X_train.columns != col].to_dict(orient='records')
  X_train = dv.fit_transform(train_dict)
  val_dict = X_val.loc[:, X_val.columns != col].to_dict(orient='records')
  X_val = dv.transform(val_dict)
  model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_val)
  acc_without_feature = (y_val == y_pred).mean()
  print("excluded:",col,round(acc_without_feature,2), "diff:" ,abs(round(orig_accuracy - acc_without_feature,2)))

excluded: lead_source 0.7 diff: 0.0
excluded: industry 0.7 diff: 0.0
excluded: number_of_courses_viewed 0.56 diff: 0.14
excluded: annual_income 0.85 diff: 0.15
excluded: employment_status 0.7 diff: 0.0
excluded: location 0.71 diff: 0.01
excluded: interaction_count 0.56 diff: 0.14
excluded: lead_score 0.71 diff: 0.01


In [33]:
X_train,X_test, y_train,y_test = train_test_split(df,y,test_size=0.2,random_state=42)
X_train,X_val, y_train,y_val = train_test_split(X_train,y_train,test_size=0.25,random_state=42)
dv = DictVectorizer(sparse=False)
train_dict = X_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)
val_dict = X_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

for c in [0.01, 0.1, 1, 10, 100]:
  model = RidgeClassifier(alpha=c,max_iter = 1000, random_state=42)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_val)
  accuracy = (y_val == y_pred).mean()
  print(c, round(accuracy,2))

0.01 0.85
0.1 0.85
1 0.85
10 0.86
100 0.84
